# Demographic Classification Using MLP

In [ ]:
import os
import io, os,sys,types
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from torch import autograd
from PIL import Image
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim
from torch.utils.data import Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch.utils.data

import random
import scipy.io

import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import seaborn as sns
import seaborn as sns
from sklearn.metrics import confusion_matrix

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import log_loss


from sklearn.metrics import classification_report

import torch.optim
import torch.utils.data
import pretrainedmodels

from pytorch_pretrained_vit import ViT

from tqdm import tqdm

# from lion_pytorch import Lion
import timm

import nni
from nni.experiment import Experiment


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
print(torch.cuda.is_available())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

### Model information

In [ ]:
root_dir = '/data/backupdata/backupdata/'
model_dir = '/data/backupdata/backupdata/model_parameters/'

Preandplus_input_categories = ['Aug_Preandplus_0_12_ROSE','Preandplus_0_12_ROSE', 'Preandplus_0_12_trainset',  'Preandplus_0_12_testset',
                              'Aug_Preandplus_01_2_ROSE', 'Preandplus_01_2_ROSE', 'Preandplus_01_2_trainset',  'Preandplus_01_2_testset']

# RWROP_input_categories = ['RWROP_image_demographic_ROSE', 'RWROP_image_demographic_trainset', 'RWROP_image_demographic_TKROSE', 'RWROP_setid_image_demographic_testset']

RWROP_input_categories = ['RWROP_image_demographic_trainROSE','RWROP_image_demographic_trainROSEE1', 'RWROP_image_demographic_trainset', 'RWROP_image_demographic_valset', 'RWROP_setid_image_demographic_testset']

Preandplus_categories = ['preandplus_0', 'preandplus_1']
RWROP_categories = ['RWROP_0', 'RWROP_1']

In [ ]:
param = {
    'model1_name' : 'ViT',
    'model2_name' : 'vgg19_bn',
    'preandplus_traindata_dir': '/data/eye_image/fold_0/Aug_stage_ROSE',
    'preandplus_valdata_dir': '/data/eye_image/fold_0/stage_testset',
    'preandplus_testdata_dir': '/data/eye_image/fold_0/stage_testset',
    'stage_traindata_dir': '/data/eye_image/fold_0/Aug_plus_ROSE',
    'stage_valdata_dir': '/data/eye_image/fold_0/plus_testset',
    'stage_testdata_dir': '/data/eye_image/fold_0/plus_testset',
    'stage3ROP_traindata_dir': '/data/eye_image/fold_0/Aug_Zone1_ROSE',
    'stage3ROP_valdata_dir': '/data/eye_image/fold_0/Zone1_testset',
    'stage3ROP_testdata_dir': '/data/eye_image/fold_0/Zone1_testset',
    'Zone1ROP_traindata_dir': '/data/eye_image/fold_4/Aug_rw_ROSE',
    'Zone1ROP_valdata_dir': '/data/eye_image/fold_4/rw_testset',
    'Zone1ROP_testdata_dir': '/data/eye_image/fold_4/rw_testset',
    'RWROP_traindata_dir': '/data/eye_image/fold_4/Aug_rw_ROSE',
    'RWROP_valdata_dir': '/data/eye_image/fold_4/rw_testset',
    'RWROP_testdata_dir': '/data/eye_image/fold_4/rw_testset',
    'train_batch_size': [16,16,16,16,16], 
    'val_batch_size': 64,
    'test_batch_size': 32,
    'num_epochs': 120,
    'warm_up_epoch': 5,
    'patient_epoch': 60,
    'h_lr': 0.0001,
    'l_lr': 0.00002,
    'initial_lr': 0.00001,
    'learning_rate': 0.0001,
    'S_S_GAP': [0.4,0.4,0.4,0.4,0.4],
    'w_momentum': 0.9,
    'w_weight_decay': 0.0001,
    'workers': 4,
    'seed': 42,
    'alpha': 0.55,
    'gamma': 1,
    'loss-weight': [torch.tensor([1.15,1.85], dtype=torch.float32),torch.tensor([1.15,1.85], dtype=torch.float32),
                   torch.tensor([1.15,1.85], dtype=torch.float32),torch.tensor([1.15,1.85], dtype=torch.float32),
                   torch.tensor([1.15,1.85], dtype=torch.float32)],
    'aucthreshold': 0.95,
    'aucboosting': 2,
}

VGG-ST

In [ ]:
class VGGA_ST_stage(nn.Module):
    def __init__(self, VGG19, ST, hidden_dim=512, n_layers=2, dropout=0.5):
        super(VGGA_ST_stage, self).__init__()
        self.vgg_transform = transforms.Compose([transforms.Resize((224,224))])
        self.vgg19_backbone = nn.Sequential(*list(VGG19.children())[1:-7])
        ST.flatten = nn.GELU()
        self.ST_backbone = ST
        self.MP = nn.MaxPool1d(4)
        self.vgg_linear3 = nn.Linear(6272,6272)
        self.vgg_linear4 = nn.Linear(6272,2048)
        self.vgg_linear5 = nn.Linear(2048,512)
        self.vgg_linear6 = nn.Linear(512,2)
        self.vgg_relu2 = nn.GELU()
        self.vgg_relu3 = nn.GELU()
        self.vgg_relu4 = nn.GELU()
        self.st_linear1 = nn.Linear(1024,1024)
        self.st_relu1 = nn.GELU()
        self.st_linear2 = nn.Linear(1024,2)

        layers = []
        for i in range(n_layers):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.GELU())
            layers.append(nn.Dropout(dropout))
        layers.append(nn.Linear(hidden_dim, 2))
        self.mlp_layers = nn.Sequential(*layers)

        
        self.linear1 = nn.Linear(1536,hidden_dim)
        self.relu1 = nn.GELU()
        self.softmax = nn.Softmax(dim=1)

        
#         ViT.fc = nn.Linear(768,768)
#         nn.init.xavier_uniform(model.fc.weight)
#         self.ViT_feature_layer = model
#         self.relu1 = nn.ReLU()
#         self.linear1 = nn.Linear(768,2)
#         nn.init.xavier_uniform(self.linear1.weight)
#         self.softmax = nn.Softmax(dim=1)
    def resize_img_size(self, x):
        for img_idx in range(x.shape[0]):
            if img_idx == 0:
                y = self.vgg_transform(x[img_idx]).unsqueeze(0)
            else:
                y = torch.cat((y, self.vgg_transform(x[img_idx]).unsqueeze(0)),0)
        return y
 
    
    def forward(self, x):
        vgg_output = self.vgg19_backbone(self.resize_img_size(x))
        vgg_output = vgg_output.view(vgg_output.shape[0],-1)
        vgg_output = self.MP(vgg_output)
        vgg_output = self.vgg_linear3(vgg_output)
        vgg_output = self.vgg_relu2(vgg_output)
        vgg_output = self.vgg_linear4(vgg_output)
        vgg_output = self.vgg_relu3(vgg_output)
        vgg_output = self.vgg_linear5(vgg_output)
        vgg_output = self.vgg_relu4(vgg_output)
        vgg_output1 = self.vgg_linear6(vgg_output)
        # vgg_output1 = self.vgg_softmax(vgg_output1)
        st_output = self.ST_backbone(x)
        st_output = self.st_linear1(st_output)
        st_output = self.st_relu1(st_output)
        st_output1 = self.st_linear2(st_output)
        # st_output1 = self.st_softmax(st_output1)
        
        output = torch.cat((vgg_output, st_output),1)
        output = self.linear1(output)
        output = self.relu1(output)
        logits = self.mlp_layers(output)
        probs = self.softmax(logits)

        return logits, vgg_output1, st_output1, probs
    

In [ ]:
# three loss
def train(model, loss_fn, optimizer, scheduler, param, folder_index, loader_train, loader_val, modal_dir):

    model.train()
    max_auc = 0

    best_epopch = 0
    patient_epoch = 0
    for epoch in range(param['num_epochs']):
        model.train()
        print('Starting epoch %d / %d' % (epoch + 1, param['num_epochs']))
    #         adjust_learning_rate(optimizer, epoch)
        epoch_loss = 0
        with torch.enable_grad():
            for t, (x, y) in enumerate(loader_train):

                x_var, y_var = x.to(device), y.to(device)
                _, scores, scores_resnet, scores_st = model(x_var)
                loss_m = loss_fn(scores, y_var)
                loss_r = loss_fn(scores_resnet, y_var)
                loss_st = loss_fn(scores_st, y_var)
                loss = param['3lossw'][0]*loss_m + param['3lossw'][1]*loss_r + param['3lossw'][2]*loss_st
                epoch_loss += loss.item()
                

                if (t + 1) % 100 == 0:
                    #print(loss.item())
                    print('t = %d, loss = %.8f' % (t + 1, loss.item()))
                optimizer.zero_grad()
                loss.backward()
        #             nn.utils.clip_grad_norm_(model.parameters(), config['w_grad_clip'])
                optimizer.step()
        # # --- Record training loss ---
        # writer.add_scalar('Loss/Train', epoch_loss, epoch)
        # # --- Record lr ---
        # current_lr = optimizer.param_groups[0]['lr']
        # writer.add_scalar('Learning Rate', current_lr, epoch)
        # # --- Record Gradient Norms ---
        # total_norm = 0.0
        # for p in model.parameters():
        #     if p.grad is not None:  # Ensure gradients exist
        #         param_norm = p.grad.data.norm(2)  # Compute L2 norm
        #         total_norm += param_norm.item() ** 2
        # total_norm = total_norm ** 0.5  # Square root of the sum of squares
        # writer.add_scalar('Gradient Norm', total_norm, epoch)

        # # --- Record Model Weights and Biases ---
        # for name, p in model.named_parameters():
        #     writer.add_histogram(f'Params/{name}', p, epoch)  # Log weights/biases
        #     if p.grad is not None:
        #         writer.add_histogram(f'Gradients/{name}', p.grad, epoch)  # Log gradients

        model.eval()
        epoch_auc = validate(model, loader_val, param['S_S_GAP'][folder_index])
        patient_epoch = patient_epoch + 1
        with torch.enable_grad():
            if epoch_auc > max_auc:
                max_auc = epoch_auc
                best_epopch = epoch
                torch.save(model.state_dict(), modal_dir)
                patient_epoch = 0
            scheduler.step(max_auc)

        if patient_epoch > param['patient_epoch']:
            print('reach patient epoch')
            break
    return max_auc     


In [ ]:
def validate(model, loader, gap):
    with torch.no_grad():
        model.eval()

        test_batch_index = 0
        for x, y in loader:

            x_var = x.to(device)
            scores, _, _, _ = model(x_var)
            _, preds = scores.data.cpu().max(1)
            if test_batch_index == 0:
                y_test = y.cpu().detach().numpy()
                y_probs = scores.cpu().detach().numpy()
                y_pred = preds.numpy()
            else:
                y_test = np.append(y_test, y.cpu().detach().numpy())
                y_probs = np.vstack((y_probs, scores.cpu().detach().numpy()))
                y_pred = np.append(y_pred, preds.cpu().detach().numpy())

            test_batch_index += 1

        # print(y_test.shape)
        y_probs = y_probs[...,1:]
        # print(y_probs.shape)
        # print(y_pred.shape)
    #     test_prob_dir = './data/eye_image/fold_0/Resnet_Aug_rwROSE_test_prob.txt'
    #     np.savetxt(test_prob_dir, y_probs ,fmt='%.10e', delimiter=" ") 
    #     np.savetxt(test_label_dir, y_test ,fmt='%.10e', delimiter=" ")

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        specificity = tn / (tn + fp)
        AUC_score = roc_auc_score(y_test, y_probs, multi_class='ovr')
        # f1 = f1_score(y_test, y_pred)
        # accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        # precision = precision_score(y_test, y_pred)
        print('AUC_score:', AUC_score)
        print('recall:', recall)
        print('specificity:', specificity)

        # writer.add_scalar('AUC/Val', AUC_score, epoch)
        # writer.add_scalar('Sensitivity/Val', recall, epoch)
        # writer.add_scalar('Specificity/Val', specificity, epoch)
        # writer.add_scalar('F1/Val', f1, epoch)
        # writer.add_scalar('precision/Val', precision, epoch)
        # writer.add_scalar('accuracy/Val', accuracy, epoch)
        
        if abs(recall - specificity) >= gap:
            return 0
        else:
            return AUC_score

In [ ]:
def test(model, loader, test_prob_dir, test_label_dir):

    with torch.no_grad():
        model.eval()
    
        for name, parameters in model.named_parameters():
            parameters.requires_grad = False

        num_correct, num_samples = 0, len(loader.dataset)

        test_batch_index = 0

        for x, y in loader:

            x_var = x.to(device)
            scores, _, _, _ = model(x_var)
            _, preds = scores.data.cpu().max(1)
            if test_batch_index == 0:
                y_test = y.cpu().detach().numpy()
                y_probs = scores.cpu().detach().numpy()
                y_pred = preds.numpy()
            else:
                y_test = np.append(y_test, y.cpu().detach().numpy())
                y_probs = np.vstack((y_probs, scores.cpu().detach().numpy()))
                y_pred = np.append(y_pred, preds.cpu().detach().numpy())

            num_correct += (preds == y).sum()
            test_batch_index += 1

        acc = float(num_correct) / num_samples
        # print(y_test.shape)
        y_probs = y_probs[...,1:]
        # print(y_probs.shape)
        # print(y_pred.shape)
    #     test_prob_dir = './data/eye_image/fold_0/Resnet_Aug_rwROSE_test_prob.txt'
        np.savetxt(test_prob_dir, y_probs ,fmt='%.10e', delimiter=" ") 
        np.savetxt(test_label_dir, y_test ,fmt='%.10e', delimiter=" ")

        print('Test accuracy: {:.2f}% ({}/{})'.format(
            100.*acc,
            num_correct,
            num_samples,
            ))

        AUC_score = roc_auc_score(y_test, y_probs, multi_class='ovr')
        AUPRC_score = average_precision_score(y_test, y_probs)
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
#         sensitivity = tp / (tp + fn)
        specificity = tn / (tn + fp)
        

        
        print('AUC:')
        print(AUC_score)
        print('AUPRC:')
        print(AUPRC_score)
        print('recall:')
        print(recall)
        print('specificity:')
        print(specificity)
        print('F1:')
        print(f1)
        print('precision:')
        print(precision)
        print('Accuracy:')
        print(accuracy)

    return AUC_score, AUPRC_score, recall, specificity, f1, precision, accuracy

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, npz_file, transform=None):
        """
        Args:
            npz_file (str): Path to the .npz file containing labels, demographic data, and file names.
            image_dir (str): Directory where the images are stored.
            transform (callable, optional): Transform to be applied to the images.
        """
        # Load the .npz file
        data = np.load(npz_file, allow_pickle=True)
        self.labels = data['label']
        self.demographics = data['demographic']
        self.file_names = data['file_name']
        self.image_dir = os.path.dirname(npz_file)
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Load image
        img_dir = os.path.join(self.image_dir, os.path.basename(self.file_names[idx]))
        image = Image.open(img_dir).convert('RGB')

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        # Get demographic data and label
        # demographic = torch.tensor(self.demographics[idx], dtype=torch.float32)
        label = torch.tensor(int(self.labels[idx]), dtype=torch.long)

        return image, label

In [ ]:
fold_AUC = []
fold_AUPRC = []
fold_Recall = []
fold_Specificity = []
fold_F1 = []
fold_Precision = []
fold_Accuracy = []
for i in range(5):
    param['RWROP_traindata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), RWROP_input_categories[0],'RWROP_filename_image_demographic_trainROSE.npz')
    param['RWROP_valdata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), RWROP_input_categories[2],'RWROP_filename_image_demographic_valset.npz')
    param['RWROP_testdata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), RWROP_input_categories[3],'RWROP_filename_image_demographic_testset.npz')
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    train_loader = torch.utils.data.DataLoader(
        CustomDataset(param['RWROP_traindata_dir'], transforms.Compose([
            transforms.Resize((256,256)),
            transforms.RandomRotation(30),
            transforms.RandomVerticalFlip(0.2),
            transforms.RandomHorizontalFlip(0.2),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=param['train_batch_size'], shuffle=True,
        num_workers=param['workers'], pin_memory=True)
    val_loader = torch.utils.data.DataLoader(
        CustomDataset(param['RWROP_valdata_dir'], transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=param['val_batch_size'], shuffle=False,
        num_workers=param['workers'], pin_memory=True)
    test_loader = torch.utils.data.DataLoader(
        CustomDataset(param['RWROP_testdata_dir'], transforms.Compose([
            transforms.Resize((256,256)),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=param['test_batch_size'], shuffle=False,
        num_workers=param['workers'], pin_memory=True)
    
    vgg19_bn = pretrainedmodels.__dict__[param['model2_name']](num_classes=1000, pretrained='imagenet')
    swin_t = timm.create_model( 'swinv2_base_window16_256' , pretrained= True , num_classes=1024)
    model = VGG_ST(vgg19_bn, swin_t, hidden_dim=512, n_layers=2, dropout=0.5)
    model.to(device)

    for name, parameters in model.named_parameters():
        if "ST_backbone" in name:
            parameters.requires_grad = False
    for name, parameters in model.named_parameters():
        if "ST_backbone.head.fc" in name:
            parameters.requires_grad = True

    criterion = nn.CrossEntropyLoss(weight=param['loss-weight'][i].to(device))
    optimizer = torch.optim.Adam(filter(lambda p:p.requires_grad, model.parameters()), lr=param['learning_rate'], weight_decay=param['w_weight_decay'])
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5,threshold=1e-5, threshold_mode='abs', patience=10, min_lr=param['initial_lr'])
    net_dir = os.path.join(model_dir, 'VGG-ST-512'+ 'fold_'+ str(i) + RWROP_input_categories[0]+'_raw_RWROP_model_parameter1.pkl')
    if os.path.exists(net_dir):
        os.remove(net_dir)
    best_auc = train(model, criterion, optimizer, scheduler, param, i, train_loader, val_loader, net_dir)
    model.load_state_dict(torch.load(net_dir))
    test_prob_dir = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-512'+RWROP_input_categories[0]+'_raw_RWROP_test_prob.txt')
    test_label_dir = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-512'+RWROP_input_categories[0]+'_raw_RWROP_test_label.txt')
    AUC_score, AUPRC_score, recall, specificity, f1, precision, accuracy = test(model, test_loader, test_prob_dir, test_label_dir)
    fold_AUC.append(AUC_score)
    fold_AUPRC.append(AUPRC_score)
    fold_Recall.append(recall)
    fold_Specificity.append(specificity)
    fold_F1.append(f1)
    fold_Precision.append(precision)
    fold_Accuracy.append(accuracy)
assert len(fold_AUC) == len(fold_AUPRC) == len(fold_Recall) == len(fold_Specificity) == len(fold_F1) == len(fold_Precision) == len(fold_Accuracy) == 5
AUC_result = np.array(fold_AUC)
AUPRC_result = np.array(fold_AUPRC)
F1_result = np.array(fold_F1)
Accuracy_result = np.array(fold_Accuracy)
Recall_result = np.array(fold_Recall)
Precision_result = np.array(fold_Precision)
# Sensitivity_result = np.array(fold_Sensitivity)
Specificity_result = np.array(fold_Specificity)

print('AUC_result')
print(AUC_result)
print('AUPRC_result')
print(AUPRC_result)
print('Recall_result')
print(Recall_result)
print('Specificity_result')
print(Specificity_result)
print('F1_result')
print(F1_result)
print('Precision_result')
print(Precision_result)
print('Accuracy_result')
print(Accuracy_result)


# print('Sensitivity_result')
# print(Sensitivity_result)


print('Avg_AUC:')
print(np.mean(AUC_result))
print('std_AUC:')
print(np.std(AUC_result))

print('Avg_AUPRC:')
print(np.mean(AUPRC_result))
print('std_AUPRC:')
print(np.std(AUPRC_result))

print('Avg_Recall:')
print(np.mean(Recall_result))
print('std_Recall:')
print(np.std(Recall_result))

print('Avg_Specificity:')
print(np.mean(Specificity_result))
print('std_Specificity:')
print(np.std(Specificity_result))

print('Avg_F1:')
print(np.mean(F1_result))
print('std_F1:')
print(np.std(F1_result))

print('Avg_Precision:')
print(np.mean(Precision_result))
print('std_Precision:')
print(np.std(Precision_result))

print('Avg_Accuracy:')
print(np.mean(Accuracy_result))
print('std_Accuracy:')
print(np.std(Accuracy_result))


    


### RandomForest

In [ ]:
image_feature_dim_list = [5,16,32,64,128,256]
n_estimators_list = [200,300,400,500]
max_depth_list = [None]
min_samples_split_list = [2, 3, 5]
min_samples_leaf_list = [1, 2, 3]
max_features_list = ['sqrt', 'log2', None]
class_weight_list = [{0: 1, 1: 1},{0: 1, 1: 2}, {0: 1, 1: 3}, {0: 1, 1: 4}]
auc_list = []
recall_list = []
specificity_list = []
f1_list = []
precision_list = []
accuracy_list = []

best_setting_result = {'fold_0': None, 'fold_1': None, 'fold_2': None, 'fold_3': None, 'fold_4': None}
for i in range(5):
    max_auc = 0
    pbar = tqdm(total=len(image_feature_dim_list) * len(n_estimators_list) * len(max_depth_list) * len(min_samples_split_list) * len(min_samples_leaf_list) * len(max_features_list) * len(class_weight_list), desc=f"Processing fold {i}")
    for image_feature_dim in image_feature_dim_list:
        param['RWROP_traindata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-'+str(image_feature_dim)+'-demograhpic'+RWROP_input_categories[0]+'_features.npz')
        param['RWROP_valdata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-'+str(image_feature_dim)+'-demograhpic'+RWROP_input_categories[3]+'_features.npz')
        train_data = np.load(param['RWROP_traindata_dir'], allow_pickle=True)
        val_data = np.load(param['RWROP_valdata_dir'], allow_pickle=True)
        train_demographic = train_data['demographic_features'][:, :5]
        train_label = train_data['label'].astype(int)
        val_demographic = val_data['demographic_features'][:, :5]
        val_image_probs = val_data['image_probs']
        val_label = val_data['label'].astype(int)
        for n_estimators in n_estimators_list:
            for max_depth in max_depth_list:
                for min_samples_split in min_samples_split_list:
                    for min_samples_leaf in min_samples_leaf_list:
                        for max_features in max_features_list:
                            for class_weight in class_weight_list:
                                rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split,
                                                       min_samples_leaf=min_samples_leaf, max_features=max_features, class_weight=class_weight, random_state=42)
                                rf.fit(train_demographic,train_label)
                                y_proba = rf.predict_proba(val_demographic)
                                y_proba_class_1 = y_proba[:, 1]
                                y_proba_class_1 = (y_proba_class_1 + val_image_probs) / 2
                                AUC_score = roc_auc_score(val_label, y_proba_class_1, multi_class='ovr')
                                pbar.update(1)
                                if AUC_score > max_auc:
                                    max_auc = AUC_score
                                    best_setting = {'image_feature_dim': image_feature_dim, 'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': min_samples_split,
                                                      'min_samples_leaf': min_samples_leaf, 'max_features': max_features, 'class_weight': class_weight}
    pbar.close()
    param['RWROP_traindata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-'+str(best_setting['image_feature_dim'])+'-demograhpic'+RWROP_input_categories[0]+'_features.npz')
    param['RWROP_testdata_dir'] = os.path.join(root_dir, 'fold_'+ str(i), 'VGG-ST-'+str(best_setting['image_feature_dim'])+'-demograhpic'+RWROP_input_categories[4]+'_features.npz')
    train_data = np.load(param['RWROP_traindata_dir'], allow_pickle=True)
    test_data = np.load(param['RWROP_testdata_dir'], allow_pickle=True)
    train_demographic = train_data['demographic_features'][:, :5]
    train_label = train_data['label'].astype(int)
    test_demographic = test_data['demographic_features'][:, :5]
    test_image_probs = test_data['image_probs']
    test_label = test_data['label']
    rf = RandomForestClassifier(n_estimators=best_setting['n_estimators'], max_depth=best_setting['max_depth'], min_samples_split=best_setting['min_samples_split'],
                               min_samples_leaf=best_setting['min_samples_leaf'], max_features=best_setting['max_features'], class_weight=best_setting['class_weight'], random_state=42)
    rf.fit(train_demographic,train_label)
    y_proba = rf.predict_proba(test_demographic)
    y_proba_class_1 = y_proba[:, 1]
    y_proba_class_1 = (y_proba_class_1 + test_image_probs) / 2
    AUC_score = roc_auc_score(test_label, y_proba_class_1, multi_class='ovr')
    y_pred_rf = np.where(y_proba_class_1 >= 0.5, 1, 0)
    f1 = f1_score(test_label, y_pred_rf)
    accuracy = accuracy_score(test_label, y_pred_rf)
    recall = recall_score(test_label, y_pred_rf)
    precision = precision_score(test_label, y_pred_rf)
    tn, fp, fn, tp = confusion_matrix(test_label, y_pred_rf).ravel()
    specificity = tn / (tn + fp)
    auc_list.append(AUC_score)
    f1_list.append(f1)
    accuracy_list.append(accuracy)
    recall_list.append(recall)
    precision_list.append(precision)
    specificity_list.append(specificity)
    best_setting_result['fold_'+ str(i)] = {'image_feature_dim': best_setting['image_feature_dim'], 'n_estimators': best_setting['n_estimators'], 'max_depth': best_setting['max_depth'], 'min_samples_split': best_setting['min_samples_split'],
                          'min_samples_leaf': best_setting['min_samples_leaf'], 'max_features': best_setting['max_features'], 'class_weight': best_setting['class_weight'],
                          'AUC': AUC_score, 'Recall': recall, 'Specificity': specificity, 'F1': f1, 'Precision': precision, 'Accuracy': accuracy}

assert len(auc_list) == len(f1_list) == len(accuracy_list) == len(recall_list) == len(precision_list) == len(specificity_list) == 5
np_auc = np.array(auc_list)
np_f1 = np.array(f1_list)
np_accuracy = np.array(accuracy_list)
np_recall = np.array(recall_list)
np_precision = np.array(precision_list)
np_specificity = np.array(specificity_list)
print('Average AUC: ', np.mean(np_auc))
print('std AUC: ', np.std(np_auc))
print('Average recall: ', np.mean(np_recall))
print('std recall: ', np.std(np_recall))
print('Average specificity: ', np.mean(np_specificity))
print('std specificity: ', np.std(np_specificity))
print('Average F1: ', np.mean(np_f1))
print('std F1: ', np.std(np_f1))
print('Average precision: ', np.mean(np_precision))
print('std precision: ', np.std(np_precision))
print('Average accuracy: ', np.mean(np_accuracy))
print('std accuracy: ', np.std(np_accuracy))
print(best_setting_result)




